## **数据清理**

In [7]:
import pandas as pd
import numpy as np
import pyreadstat

class DataCleaner:
    def __init__(self, input_dir='data/data_cl1/', output_dir='data/data_cl2/'):
        self.input_dir = input_dir
        self.output_dir = output_dir
        self.data1 = None
        self.data2 = None
        self.data3 = None
        self.data4 = None
        self.data5 = None
        
        # 列名映射字典
        self.column_mapping = {
            'ba001': 'sex',
            'ba008': 'residence', 
            'ba009': 'tor',
            'ba010': 'edu',
            'ba011': 'marriage',
            'ba014': 'pension insurance',
            'ba015': 'toi',
            'xrage_x': 'age',
            'zredu': 'edu_level',
            'fa001': 'erg',
            'fa004': 'retirement',
            'fc002': 'farm work',
            'fc034': 'fa004',
            'fc042_s5': 'type of work',
            'ff001': 'salary_x',
            'ff004': 'fuli',
            'ff004_1': 'ga001',
            'ff004_2': 'salary_y',
            'fh001': 'health',
            'fh002': 'chronic_ill',
            'xmainjobdays': 'xfagepossibility',
            'xhhldfarmdays': 'xchildalivenum',
            'xrage_y': 'age2',
            'da001': 'health_status',
            'da003': 'chronic_disease_count',
            'ca017': 'fsfc',
            'ca018': 'fsff',
            'iit': 'iit'
        }
    
    def load_demographic_background(self):
        """加载并处理个人信息数据"""
        self.data1, meta1 = pyreadstat.read_dta(f'{self.input_dir}Demographic_Background.dta')
        columns1 = ["ID", "householdID", "ba001", "ba008", "ba009", "ba010", "ba011", "ba014", "ba015", "xrage", "zredu"]
        self.data1 = self.data1[columns1]
        
        # 重命名列
        self.data1 = self.data1.rename(columns={
            'ba001': 'sex',
            'ba008': 'residence', #住哪
            'ba009': 'tor',  #户口类型
            'ba010': 'edu',
            'ba011': 'marriage',
            'ba014': 'pension_insurance',
            'ba015': 'toi',
            'xrage': 'age',
            'zredu': 'edu_level'
        })
        
        return self.data1
    
    def load_family_information(self):
        """加载并处理家庭信息数据"""
        self.data2, meta2 = pyreadstat.read_dta(f'{self.input_dir}Family_Information.dta')
        
        # 选择需要的列
        columns2 = ["householdID", "xchildalivenum", "ca008_1_", "ca008_2_", "ca008_3_", "ca008_4_", "ca008_5_", 
                   "ca008_6_", "ca008_7_", "ca008_8_", "ca008_9_", "ca008_10_", "ca008_11_", "ca008_12_", 
                   "ca008_13_", "ca008_14_", "ca008_15_", "ca008_16_", "ca008_17_",
                   "ca017_1_1_", "ca017_1_2_", "ca017_1_3_", "ca017_1_4_", "ca017_1_5_", "ca017_1_6_", 
                   "ca017_1_7_", "ca017_1_8_", "ca017_1_9_", "ca017_1_10_", "ca017_1_11_", "ca017_1_12_", 
                   "ca017_1_13_", "ca017_1_14_", "ca017_1_15_", "ca017_1_16_", "ca017_1_17_",
                   "ca018_1_1_", "ca018_1_2_", "ca018_1_3_", "ca018_1_4_", "ca018_1_5_", "ca018_1_6_", 
                   "ca018_1_7_", "ca018_1_8_", "ca018_1_9_", "ca018_1_10_", "ca018_1_11_", "ca018_1_12_", 
                   "ca018_1_13_", "ca018_1_14_", "ca018_1_15_", "ca018_1_16_", "ca018_1_17_"]
        
        self.data2 = self.data2[columns2]
        
        # 计算汇总列
        self.data2['fsfc'] = self.data2.loc[:, 'ca017_1_1_':'ca017_1_17_'].sum(axis=1)
        self.data2['fsff'] = self.data2.loc[:, 'ca018_1_1_':'ca018_1_17_'].sum(axis=1)
        
        # 计算ca008各类别的计数
        c = ["ca008_1_", "ca008_2_", "ca008_3_", "ca008_4_", "ca008_5_", "ca008_6_", "ca008_7_", 
             "ca008_8_", "ca008_9_", "ca008_10_", "ca008_11_", "ca008_12_", "ca008_13_", "ca008_14_", 
             "ca008_15_", "ca008_16_", "ca008_17_"]
        
        self.data2['ca0081'] = (self.data2[c] == 1).sum(axis=1)
        self.data2['ca0082'] = (self.data2[c] == 2).sum(axis=1)
        self.data2['ca0083'] = (self.data2[c] == 3).sum(axis=1)
        self.data2['ca0084'] = (self.data2[c] == 4).sum(axis=1)
        
        # 计算iit (代际经济支持净值)
        self.data2['iit'] = self.data2['fsff'] - self.data2['fsfc']
        
        # 选择最终需要的列
        c_new = ["householdID", "xchildalivenum", "fsfc", "fsff", "iit", "ca0081", "ca0082", "ca0083", "ca0084"]
        self.data2 = self.data2[c_new]
        
        return self.data2
    
    def load_health_status(self):
        """加载并处理健康状况数据"""
        self.data3, meta3 = pyreadstat.read_dta(f'{self.input_dir}Health_Status_and_Functioning.dta')
        
        columns3 = ["ID", "householdID", "da001", "db013", "xfagepossibility", "da003_1_", "da003_2_", 
                   "da003_3_", "da003_4_", "da003_5_", "da003_6_", "da003_7_", "da003_8_", "da003_9_",
                   "da003_10_", "da003_11_", "da003_12_", "da003_13_", "da003_14_", "da003_15_"]
        
        self.data3 = self.data3[columns3]
        
        # 计算慢性病数量
        c1 = ["da003_1_", "da003_2_", "da003_3_", "da003_4_", "da003_5_", "da003_6_", "da003_7_", 
              "da003_8_", "da003_9_", "da003_10_", "da003_11_", "da003_12_", "da003_13_", "da003_14_", "da003_15_"]
        
        self.data3['chronic_ill'] = (self.data3[c1] == 1).sum(axis=1)
        
        # 重命名列
        self.data3 = self.data3.rename(columns={
            'da001': 'health',
            'xfagepossibility': 'xfagepossibility'
        })
        
        # 选择最终需要的列
        self.data3 = self.data3[["ID", "householdID", "health", "chronic_ill", "xfagepossibility"]]
        
        return self.data3
    
    def load_individual_income(self):
        """加载并处理个人收入数据"""
        self.data4, meta4 = pyreadstat.read_dta(f'{self.input_dir}Individual_Income.dta')
        columns4 = ["ID", "householdID", "ga001", "ga002"]
        self.data4 = self.data4[columns4]
        
        # 重命名列
        self.data4 = self.data4.rename(columns={
            'ga001': 'has_income',
            'ga002': 'salary'
        })
        
        return self.data4
    
    def load_work_retirement(self):
        """加载并处理工作与退休数据"""
        self.data5, meta5 = pyreadstat.read_dta(f'{self.input_dir}Work_Retirement.dta')
        columns5 = ["ID", "householdID", "fa001", "fa004", "fc002", "fc034", "fc042_s5", 
                   "ff001", "ff004", "ff004_1", "ff004_2", "fh001", "fh002",
                   "xmainjobdays", "xhhldfarmdays", "xrage"]
        self.data5 = self.data5[columns5]
        
        # 重命名列
        self.data5 = self.data5.rename(columns={
            'fa001': 'employment_status',
            'fa004': 'retirement_status',
            'fc002': 'farm_work',
            'fc034': 'work_type',
            'fc042_s5': 'monthly_salary',
            'ff001': 'welfare',
            'ff004': 'pension_type', 
            'ff004_1': 'pension_amount',
            'ff004_2': 'other_income',
            'fh001': 'health_insurance',
            'fh002': 'chronic_disease',
            'xmainjobdays': 'main_job_days',
            'xhhldfarmdays': 'farm_days',
            'xrage': 'current_age'
        })
        
        return self.data5
    
    def create_willingness_feature(self, df):
        """创建延迟退休意愿特征"""
        df['will_delay_retire'] = 0
        df.loc[(df['age'] < 60) & (df['health'] <= 2), 'will_delay_retire'] = 1
        
        return df
    
    def merge_all_data(self):
        """合并所有数据"""
        # 加载所有数据
        self.load_demographic_background()
        self.load_family_information()
        self.load_health_status()
        self.load_individual_income()
        self.load_work_retirement()
        
        # 合并数据
        merged_df1 = pd.merge(self.data1, self.data5, on=['ID', 'householdID'])
        merged_df2 = pd.merge(merged_df1, self.data4, on=['ID', 'householdID'])
        merged_df3 = pd.merge(merged_df2, self.data3, on=['ID', 'householdID'])
        merged_df4 = pd.merge(merged_df3, self.data2, on=['householdID'])
        
        # 创建延迟退休意愿特征
        final_data = self.create_willingness_feature(merged_df4)
        
        return final_data
    
    def clean_and_save(self):
        """执行完整的数据清理流程并保存最终结果"""
        print("开始数据清理流程...")
        
        # 合并所有数据
        final_data = self.merge_all_data()
        
        # 保存最终结果
        output_file = f'{self.output_dir}Willingness_of_delay_retirement.xlsx'
        final_data.to_excel(output_file, index=False)
        
        print(f"数据清理完成！最终文件已保存至: {output_file}")
        print(f"最终数据集形状: {final_data.shape}")
        print(f"列名: {list(final_data.columns)}")
        
        return final_data

In [13]:
cleaner = DataCleaner()
final_data = cleaner.clean_and_save()

开始数据清理流程...
数据清理完成！最终文件已保存至: data/data_cl2/Willingness_of_delay_retirement.xlsx
最终数据集形状: (19339, 39)
列名: ['ID', 'householdID', 'sex', 'residence', 'tor', 'edu', 'marriage', 'pension_insurance', 'toi', 'age', 'edu_level', 'employment_status', 'retirement_status', 'farm_work', 'work_type', 'monthly_salary', 'welfare', 'pension_type', 'pension_amount', 'other_income', 'health_insurance', 'chronic_disease', 'main_job_days', 'farm_days', 'current_age', 'has_income', 'salary', 'health', 'chronic_ill', 'xfagepossibility', 'xchildalivenum', 'fsfc', 'fsff', 'iit', 'ca0081', 'ca0082', 'ca0083', 'ca0084', 'will_delay_retire']


In [14]:
columns = ['will_delay_retire', "sex", "edu", "edu_level", "marriage", "age", "salary",
           "tor", "health", "chronic_ill", "fsfc", "fsff", "xchildalivenum"]
final_data = final_data[columns]
final_data.head()

,will_delay_retire,sex,edu,edu_level,marriage,age,salary,tor,health,chronic_ill,fsfc,fsff,xchildalivenum
0,0,1,6,NaN,1,62,26400,2,1,0,1000,0,1
1,0,2,6,NaN,1,60,NaN,2,3,0,1000,0,1
2,0,1,NaN,4,1,64,NaN,3,NaN,0,0,0,1
3,0,2,NaN,4,1,64,NaN,3,4,0,0,0,1
4,0,2,5,NaN,4,65,NaN,2,3,2,0,0,1


In [15]:
# 1. sex转换: 1变1, 2变0
final_data['sex'] = final_data['sex'].replace({1: 1, 2: 0})

# 2. edu1 = edu + edu_level (处理NaN值)
# 先将NaN替换为0，然后相加
final_data['edu'] = final_data['edu'].fillna(0)
final_data['edu_level'] = final_data['edu_level'].fillna(0)
final_data['edu1'] = final_data['edu'] + final_data['edu_level']
final_data = final_data.drop(['edu', 'edu_level'], axis=1)
final_data['edu1'] = final_data['edu1'].replace({1: 1, 2: 2, 3: 2, 4: 2, 5: 3, 6: 3, 7: 3,
                                                8:4, 9:4, 10:4, 11:4})

# 3. salary NaN=0
final_data['salary'] = final_data['salary'].fillna(0)
final_data['log_salary'] = np.where(final_data['salary'] > 0, 
                                  np.log(final_data['salary']), 
                                  0)
final_data = final_data.drop(['salary'], axis=1)

# 4.marriage
final_data['marriage'] = final_data['marriage'].replace({1: 1, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0})

# 5.health - 修正：应该是health列而不是marriage列
final_data['health'] = final_data['health'].replace({1: 5, 2: 4, 3: 3, 4: 2, 5: 1})

print("所有数据处理完成！")
print(final_data[['will_delay_retire', 'sex', 'edu1', 'marriage', 'age', 'log_salary', 'tor', 'health', 'chronic_ill', 'fsfc', 'fsff', 'xchildalivenum']].head())


所有数据处理完成！
   will_delay_retire  sex  edu1  marriage  age  log_salary  tor  health  \
0                  0    1     3         1   62   10.181119    2     5.0   
1                  0    0     3         1   60    0.000000    2     3.0   
2                  0    1     2         1   64    0.000000    3     NaN   
3                  0    0     2         1   64    0.000000    3     2.0   
4                  0    0     3         0   65    0.000000    2     3.0   

   chronic_ill  fsfc fsff xchildalivenum  
0            0  1000    0              1  
1            0  1000    0              1  
2            0     0    0              1  
3            0     0    0              1  
4            2     0    0              1  


C:\Projects\Temp\ipykernel_25652\966861753.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_data['edu'] = final_data['edu'].fillna(0)
C:\Projects\Temp\ipykernel_25652\966861753.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_data['edu_level'] = final_data['edu_level'].fillna(0)
C:\Projects\Temp\ipykernel_25652\966861753.py:14: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `p

In [16]:
final_data.to_excel("data/data_cl2/final_data.xlsx")